#Section 0: Imports / Installs / Mounting Google Drive / Tensorboard Setup

In [0]:
import os
import shutil

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision import transforms
import torch.optim as optim

In [0]:
from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
%load_ext tensorboard
from torch.utils.tensorboard import SummaryWriter
#be sure to rename filepath as appropriatef
BASE_PATH = '/content/drive/My Drive/CIS_522_Project/'
ROOT_LOG_DIR = "{0}logs".format(BASE_PATH)

logger_3a = SummaryWriter(os.path.join(ROOT_LOG_DIR, "3a"))
logger_3b = SummaryWriter(os.path.join(ROOT_LOG_DIR, "3b"))
logger_3c = SummaryWriter(os.path.join(ROOT_LOG_DIR, "3c"))
logger_3d = SummaryWriter(os.path.join(ROOT_LOG_DIR, "3d"))
logger_4a = SummaryWriter(os.path.join(ROOT_LOG_DIR, "4a"))
logger_4b = SummaryWriter(os.path.join(ROOT_LOG_DIR, "4b"))

%tensorboard --logdir {ROOT_LOG_DIR.replace(" ", "\\ ")}

#Section 1: Data Loading / Cleaning / Processing / Creating train test splits

## 1A. FairFace

Now we are going to load the Fairface dataset from a zipped file. The images and labels (formatted in .csv) can be downloaded from https://github.com/joojs/fairface, but it is also located in the Google Drive.

In [0]:
# unzip the image .zip file, this takes about half an hour
#be sure to rename filepath as appropriate
!7z x '/content/drive/My Drive/CIS_522_Project/Fairface/fairface.zip'

Next, we are going to manipulate the file names. This is a tricky part such that without the procedure, there will be a mislabeling of the dataset. Originally the data is ordered like 1.jpg, 2.jpg, ...., 86744.jpg. However, after loading to Colab, the order is changed to 1.jpg, 10,jpg, 100.jpg, ...., 86744.jpg. To handle this, we add zeros to each name so it changes to 00001.jpg, 00002.jpg, 00003.jpg, ...., 86744.jpg and preserve the original order.

In [0]:
# change the image name to make sure the order is correct 
# Originally we found the order is "1.jpg, 10,jpg, 100.jpg, ....", which does not match the order in label file
path = '/content/fairface_o/train/train'
for filename in os.listdir(path):
    num, after = filename.split('.')
    num = num.zfill(5)
    new_filename = num + ".jpg"
    os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

path = '/content/fairface_o/val/val'
for filename in os.listdir(path):
    num, after = filename.split('.')
    num = num.zfill(5)
    new_filename = num + ".jpg"
    os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

Then we load the labels, which includes gender and ethnicity, and define the imagefolder.

In [0]:
# read label file
#be sure to rename filepath as appropriate
train_label = pd.read_csv("/content/drive/My Drive/CIS_522_Project/Fairface/fairface_label_train.csv")
val_label = pd.read_csv("/content/drive/My Drive/CIS_522_Project/Fairface/fairface_label_val.csv")

train_label['race'] = train_label['race'].str.replace('_Hispanic', '')
val_label['race'] = val_label['race'].str.replace('_Hispanic', '')
train_label['race'] = train_label['race'].str.replace(' ', '')
val_label['race'] = val_label['race'].str.replace(' ', '')

train_gender = np.argmax(pd.get_dummies(train_label['gender']).to_numpy(),axis=1)
val_gender = np.argmax(pd.get_dummies(val_label['gender']).to_numpy(),axis=1)

train_race = np.argmax(pd.get_dummies(train_label['race']).to_numpy(),axis=1)
val_race = np.argmax(pd.get_dummies(val_label['race']).to_numpy(),axis=1)


# create Dataset/Dataloader for training and validation set
train_root = '/content/fairface_o/train' 
val_root = '/content/fairface_o/val' 

fairface_data_train = ImageFolder(root = train_root,transform = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()]))
fairface_data_val = ImageFolder(root = val_root,transform = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()]))

Noted are the numerical labels for ethnicity and gender:

Ethnicity:
0. Black
1. East Asian
2. Indian
3. Latino
4. Middle Eastern
5. Southeast Asian
6. White

Gender:
0. Female
1. Male

##1B. IMDB

The following code uses the cleaned excel document *imdb.xlsx* as well as a folder of appropriately named images under the filepath, e.g. .../imdb/test/firstname_lastname_0. However, the raw IMDB images and labels (in .mat format) can be found at https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/, under links for "metadata" and "faces only."

In [0]:
#be sure to rename filepath as appropriate
!unzip '/content/drive/My Drive/CIS_522_Project/imdb.zip'

In [0]:
src = '/content/imdb/test'
os.remove('/content/imdb/test/.DS_Store.jpg')

#be sure to rename filepath as appropriate
df = pd.read_excel('/content/drive/My Drive/CIS_522_Project/imdb.xlsx')
df = df.sort_values(by=['full_path'])

In [0]:
test_gender = np.argmax(pd.get_dummies(df.gender).to_numpy(),axis=1)
test_race = np.argmax(pd.get_dummies(df.ethnicity).to_numpy(),axis=1)

images = ImageFolder(root = '/content/imdb',transform = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()]))

#Section 2: Dataset / DataLoader Setup 

We next define the Dataset we will use for both FairFace and IMDB. The dataset contains race and gender information for the data (each image).

In [0]:
class Face_Dataset(Dataset):
    def __init__(self, data, race, gender, transform = None):
      self.data = data
      self.race = race
      self.gender = gender
      self.transform = transform     
      pass

    def __len__(self):
      return len(self.data)
      
    def __getitem__(self, idx):
      if torch.is_tensor(idx):
            idx = idx.tolist()
      item_data = self.data[idx][0]
      item_race = self.race[idx]
      item_gender = self.gender[idx]

      return item_data, item_race, item_gender

This function visalizes 5 images from a dataset.

In [0]:
def visualize_images(data):
  ethnicities = ['Black','East Asian','Indian','Latino','Middle Eastern','Southeast Asian','White']
  for i in range(5):
    fig = plt.figure()
    inputs, gender_label, race_label = data[i][0], data[i][2], data[i][1]
    if gender_label == 0:
      gender_label = 'Female'
    else:
      gender_label = 'Male'
    
    race_label = ethnicities[race_label]

    plt.title('Gender is {label1}, Race is {label2}'.format(label2=race_label, label1=gender_label))
    plt.imshow(inputs.permute(1,2,0).cpu().numpy(), cmap='gray')
    plt.show()

##2A. FairFace

In [0]:
# create the dataset
train_data = Face_Dataset(fairface_data_train, train_race, train_gender)
val_data = Face_Dataset(fairface_data_val, val_race, val_gender)

# construct dataloaders
b_size = 128
train_loader = DataLoader(train_data, batch_size=b_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=b_size, shuffle=True)

In [0]:
visualize_images(train_data)

In [0]:
visualize_images(val_data)

##2B. IMDB

In [0]:
test_data = Face_Dataset(images, test_race, test_gender)
test_loader = DataLoader(test_data, batch_size=b_size)

In [0]:
visualize_images(test_data)

#Section 3: Network Model / Network Initialization / Loss and Training / Validation / Testing Loop for Baseline Models

##Training/Testing loops
We created generalized training and testing loops that can apply to nearly all of our models.

In [0]:
def test_model(test_dataloader, net, lossfn, fairmasking=False):
    total_loss = 0
    correct = 0
    total = 0  
    correct_lst = [0] * 7
    total_lst = [0] * 7
    net.eval()
    with torch.no_grad():

        for image, race, gender in test_dataloader:
            image, race, gender = image.to(device), race.to(device), gender.to(device)

            if fairmasking == True:
              output, _ = net(image)
            else:
              output = net(image)

            loss = lossfn(output, gender)
            _, pred = torch.max(output.data, 1)

            total_loss += loss.item()
            total += race.shape[0]

            correct += (torch.sum(pred == gender)).item()

            for i in range(7):
              correct_i = correct_lst[i]
              total_i = total_lst[i]
              correct_i += (torch.sum((pred == gender) & (race == i))).item()
              total_i += (torch.sum(race == i)).item()
              correct_lst[i] = correct_i
              total_lst[i] = total_i

        acc = correct/total
        acc0 = correct_lst[0]/total_lst[0]
        acc1 = correct_lst[1]/total_lst[1]
        acc2 = correct_lst[2]/total_lst[2]
        acc3 = correct_lst[3]/total_lst[3]
        acc4 = correct_lst[4]/total_lst[4]
        acc5 = correct_lst[5]/total_lst[5]
        acc6 = correct_lst[6]/total_lst[6]

        var = np.var([acc0, acc1, acc2, acc3, acc4, acc5, acc6])

        print('Evaluation accuracy on test set: %s, Loss: %s' %(acc, total_loss/total))
        print('Unfairness:', var)
        print('Gender accuracy|Black:',acc0)
        print('Gender accuracy|East Asian:',acc1)
        print('Gender accuracy|Indian:',acc2)
        print('Gender accuracy|Latino:',acc3)
        print('Gender accuracy|Middle Eastern:',acc4)
        print('Gender accuracy|Southeast Asian:',acc5)
        print('Gender accuracy|White:',acc6)
        
        return acc

In [0]:
def train_model(train_dataloader, val_dataloader, net, optimizer, lossfn, logger=None, epochs=10, verbose=True, print_every=10, fairmasking=False):     
    net = net.to(device)
    net.train()
    
    step = 0
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for image, race, gender in train_dataloader:
            iamge, race, gender = image.to(device), race.to(device), gender.to(device)
            optimizer.zero_grad()
            if fairmasking == True:
              output, _ = net(image)
            else:
              output = net(image)

            loss = lossfn(output, gender)
            loss.backward()
            optimizer.step()

            _, pred = torch.max(output, 1)                    
            total += race.shape[0]
            total_loss += loss.item()

            correct += (torch.sum(pred == gender)).item()

            if ((step % print_every) == 0):
                if logger != None:
                  info = { ('loss') : loss.item()}
                  for tag, value in info.items():
                    logger.add_scalar(tag, value, step)

                  logger.add_scalar('train accuracy', acc, step)
                  val_acc = test_model(val_dataloader, net, lossfn, fairmasking)
                  logger.add_scalar('val accuracy', val_acc, step)

                if verbose:
                    print(" --- step: %s Acc: %s Loss: %s " %(step, correct/total, total_loss/total) )
            
            step += 1

        print("Training: Epoch: %s, Acc: %s, Loss: %s" %(epoch, correct/total, total_loss/total))
        scheduler.step()

##3A. Logistic Regression

In [0]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size*input_size*3, output_size)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = F.softmax(self.linear(x))
        return out 

In [0]:
# training Logistic baseline
input_size = 256
output_size = 2
learning_rate = 0.001     
num_epochs = 30

loss_function = nn.CrossEntropyLoss()
log_reg = LogisticRegression()
optimizer = optim.SGD(log_reg.parameters(), lr = learning_rate)

train_model(train_loader, val_loader, log_reg, optimizer, loss_function,  logger_3a, num_epochs)

In [0]:
test_model(val_loader, log_reg, loss_function)

##3B. FeedForward

In [0]:
input_size = 256
output_size = 2

class FeedForward(nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.fc = nn.Sequential(nn.Dropout(p=0.2),
                                nn.Linear(input_size*input_size*3, 500), 
                                nn.ReLU(), 
                                nn.BatchNorm1d(500),
                                nn.Linear(500, 250), 
                                nn.ReLU(),                            
                                nn.BatchNorm1d(250),
                                nn.Linear(250, output_size))

    def forward(self, x):
        x = x.view(x.size(0), -1)
        out= self.fc(x)
        return out

In [0]:
feed_forward = FeedForward()
l2 = 0.001
num_epochs = 20
loss1 = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(feed_forward.parameters(), lr =l2)
train_model(train_loader, val_loader, feed_forward, optimizer, loss_function,  logger_3b, num_epochs)

In [0]:
test_model(val_loader, feed_forward, loss_function)

##3C. CNN

In [0]:
# Defining the model
class View(nn.Module):
    def __init__(self,o):
        super().__init__()
        self.o = o

    def forward(self,x):
        return x.view(-1, self.o)
    
class CNN(nn.Module):
    def __init__(self, cin=3, c1=16, c2=32, c3=128):
        super().__init__()
        d = 0.5

        def convbn(ci,co,ksz,s=1,pz=0):
            return nn.Sequential(
                nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
                nn.BatchNorm2d(co),
                nn.ReLU(True),
                )

        self.m = nn.Sequential(
            convbn(cin,c1,4,2,1),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size = 2), 
            convbn(c1,c2,4,2,1),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size = 2), 
            convbn(c2,c3,3,1,1),
            nn.AvgPool2d(2),
            )
        self.f = nn.Sequential(nn.Linear(2048*4, 2)
                                    )
    def forward(self, x):
        x = self.m(x)
        x = x.view(x.size(0),-1)
        out = self.f(x)
        return out 

We train the CNN using Adam optimizer with default learning rate (1e-03) and 10 epochs. We also set a scheduler for the optimizer, multiply learning rate by 0.9 after each epoch. As a result, The training accuracy achieves 89.55% and validation accuracy is 85.63%.

In [0]:
cnn = CNN()
learning_rate = 0.001
optimizer = optim.Adam(cnn.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()
num_epochs = 10

train_model(train_loader, val_loader, cnn, optimizer, loss_function,  logger_3c, num_epochs)

In [0]:
test_model(val_loader, cnn, loss_function)

##3D. ResNet

In [0]:
def conv3x3(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size= 3, stride = stride, 
                     padding = 1, bias = False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        
        return out

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 7):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace = True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(4096, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride = 1):
        downsample = None
        if (stride !=1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride = stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))

        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.relu(self.bn(out))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

In [0]:
# net_args current ResNet34
net_args = {
    "block": ResidualBlock,
    "layers": [3, 4, 6, 3]
}

resnet = ResNet(**net_args)
learning_rate = 0.0001
optimizer = optim.Adam(resnet.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()
num_epochs = 30

train_model(train_loader, val_loader, resnet, optimizer, loss_function,  logger_3d, num_epochs)

In [0]:
test_model(val_loader, resnet, loss_function)

#Section 4: Network Model / Network Initialization / Loss and Training / Validation / Testing Loop for Fair Models

##4A. Fair regularization of ResNet

### Penalty function and weight decay

We regularize by putting penalties on differences on loss/accuracy among different races. We basically use $L^2$ penalties, i.e.
$$
penalty_{total} = \sum_{i=1}^{7} (averageloss_{race_i} - averageloss_{all})^2
$$.

We also use weight decay througout the training, e.g. we started from weight $\epsilon = 1$ in the first epoch and decay it to 0.001 at the the of the training.

In [0]:
def loss_penalty(total_loss, black_loss, eastasian_loss, indian_loss,
                 latino_loss, middleeastern_loss, southeastasian_loss, white_loss):
    loss = (total_loss - black_loss)**2 + (total_loss - eastasian_loss)**2 + (total_loss- indian_loss)**2 + (total_loss-latino_loss)**2 + 10*(total_loss - middleeastern_loss)**2 + (total_loss - southeastasian_loss)**2 + (total_loss - white_loss)**2
    loss *= 10
    return loss

In [0]:
# Penalty function on accuracy difference across features
def acc_penalty(total_acc, black_acc, eastasian_acc, indian_acc,
                 latino_acc, middleeastern_acc, southeastasian_acc, white_acc): 
    loss = (total_acc - black_acc)**2 + (total_acc - eastasian_acc)**2 + (total_acc- indian_acc)**2 + (total_acc-latino_acc)**2 + 10*(total_acc - middleeastern_acc)**2 + (total_acc - southeastasian_acc)**2 + (total_acc - white_acc)**2
    loss *= 10
    return loss

In [0]:
# loss decay
epsilon_start = 1
epsilon_end = 0.01
def decay_rate(steps, total_epoch):
    return epsilon_start - (steps/(total_epoch*1400)) * (epsilon_start - epsilon_end)

###Training loop

In [0]:
def train_penalized_model(train_dataloader, val_dataloader, net, optimizer, lossfn, logger=None, epochs = 20, verbose = True, print_every=100, mode="loss"):
    step = 0

    net.to(device)
    net.train() 

    # for 5 epochs
    for epoch in range(epochs):
        total_loss = 0
    
        correct = 0
        total = 0

        correct_lst = [0] * 7
        total_lst = [0] * 7
        
        for data, race, gender in train_dataloader:
            loss_total_lst = [0] * 7

            data, race, gender = data.to(device), race.to(device), gender.to(device)

            optimizer.zero_grad()
            output = net(data)

            _, pred = torch.max(output, 1)
            
            correct += (torch.sum(pred == gender)).item()
            total += race.shape[0]

            for i in range(7):
              correct_i = correct_lst[i]
              total_i = total_lst[i]
              loss_tot_i = loss_tot_lst[i]

              correct_i += (torch.sum((pred == gender) & (race == i))).item()
              total_i += (torch.sum(race == i)).item()
              loss_i = lossfn(torch.index_select(output, 0, 1*(race == 0)), torch.index_select(gender,0, 1*(race == 0)))
              loss_total_i += loss_i.item()
              
              correct_lst[i] = correct_i
              total_lst[i] = total_i
              loss_total_lst[i] = loss_tot_i


            acc = correct/total
            loss = lossfn(output, gender)

            total_loss_avg = loss.sum().item()/gender.shape[0]
            
            if mode == "loss":
                penalty = loss_penalty(total_loss_avg, loss_total_lst[0]/correct_lst[0], loss_total_lst[1]/correct_lst[1], loss_total_lst[2]/correct_lst[2],loss_total_lst[3]/correct_lst[3], loss_total_lst[4]/correct_lst[4], loss_total_lst[5]/correct_lst[5], loss_total_lst[6]/correct_lst[6])
            else:
                penalty = acc_penalty(acc, correct_lst[0]/total_lst[0], correct_lst[1]/total_lst[1], correct_lst[2]/total_lst[2],correct_lst[3]/total_lst[3], correct_lst[4]/total_lst[4], correct_lst[5]/total_lst[5], correct_lst[6]/total_lst[6])

            # decay the penalty
            penalty *= decay_rate(step,epochs)
            loss += penalty
            
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if ((step % print_every) == 0):
                if logger != None:
                  info = { ('loss') : loss.item()}
                  for tag, value in info.items():
                    logger.add_scalar(tag, value, step)
                    
                  logger.add_scalar('train accuracy', acc, step)
                  val_acc = test_model(val_dataloader, net, lossfn, fairmasking)
                  logger.add_scalar('val accuracy', val_acc, step)
                if verbose:
                    print(" --- step: %s Acc: %s Loss: %s " %(step, correct/total, total_loss/total) )

            step += 1

In [0]:
net_args = {
    "block": ResidualBlock,
    "layers": [3, 4, 6, 3]
}

resnet_reg = ResNet(**net_args).to(device)
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(resnet_reg.parameters(), lr = learning_rate)
num_epochs = 30

train_penalized_model(train_loader, val_loader, resnet_reg, optimizer, loss_function, logger_4a, num_epochs)

In [0]:
test_model(val_loader, resnet_reg, loss_function)

##4B. Fair masking

In this section, we propose to use fair masking to mitigate the unfairness of neural networks in Fairface dataset. The general idea of masking is that for a network(e.g. CNN, ResNet), we will choose one hidden layer to add fairness regularization. To do this we will output the values of all features in this hidden layer, mask out those that are strongly dependent with the protected feature. In this framework, it can be tricky to quantify the dependence between hidden features and protected features since the former is continuous while the latter is discrete in most cases (e.g. gender, ethnicity). Actually, there aren't any widely-used dependence test between a continuous variable and a non-binary categorical varible.

To conduct this dependence test, we propose two approaches: Correlation-based dependence test and logistic regression based dependence test.


### Model structure

In [0]:
# implement two sequential networks together
# redefine the networks now :)
class Seq_CNN(nn.Module):
    def __init__(self, cin=3, c1=16, c2=32, c3=128):
        super().__init__()
        d = 0.5

        def convbn(ci,co,ksz,s=1,pz=0):
            return nn.Sequential(
                nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
                nn.BatchNorm2d(co),
                nn.ReLU(True),
                )
        
        self.mask = torch.tensor(np.ones(32))
        self.latent_feature = torch.tensor(np.zeros(32))

        self.seq_1 = nn.Sequential(
            convbn(cin,c1,4,2,1),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size = 2), #64*64
            convbn(c1,c2,4,2,1),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size = 2), #16*16
            convbn(c2,c3,3,1,1),
            nn.AvgPool2d(2),
            )
        
        self.f = nn.Linear(2048*4, 32)
        
        self.seq_2 = nn.Sequential(
            nn.Linear(32, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(500, 2)
        )


    def forward(self, x):
        # first part
        x_h = self.seq_1(x)
        x_h = x_h.view(x_h.size(0),-1)
        latent_feature = self.f(x_h).to(device)
        mask = latent_feature * self.mask.to(device)
        # second part
        out = self.seq_2(mask.float())
        return out, latent_feature
    
    # drop out some features
    def Penalize(self, index):
        for i in range(len(self.mask)):
          if i in index:
            self.mask[i] = 0
        return None

    def Recover(self):
        for i in range(len(self.mask)):
            self.mask[i] = 1
        return None

In [0]:
seq_cnn = Seq_CNN()
learning_rate = 0.001
optimizer = optim.Adam(seq_cnn.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()
num_epochs = 10

train_model(train_loader, val_loader, seq_cnn, optimizer, loss_function, logger_4b, num_epochs, fairmasking=True)

In [0]:
test_model(val_loader, seq_cnn, loss_function, fairmasking=True)

### Correlation-based Masking

After training, we can apply rank the 7 ethnicity groups according to their gender classification accuracy on validation set. Then we can calculate the correlation between each feature and the ranking of ethnicity. For those that has correlation larger than prespecified threshold, we will mask out them by setting values to be 0.

In [0]:
def pearsonr(x, y):
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    r_num = xm.dot(ym)
    r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
    return r_num / r_den

In [0]:
def penalize_model_cor(train_dataloader, net, threshold):
    # at the end add penalty
    total = train_gender.shape[0]
    hidden_feature_count = torch.zeros([total,32])
    protected_count = torch.zeros(total)
    count = 0
    Order = np.array([6,5,3,2,0,4,3])
    for i, (data, race, gender) in enumerate(train_dataloader):
            data, race, gender = data.to(device), race.to(device), gender.to(device)
            output,hidden_feature = net(data)

            for j in range(data.shape[0]):
              H_feature = Variable(hidden_feature[j],requires_grad = False)
              hidden_feature_count[count,:] = H_feature
              Race = Order[race[j].cpu().int()]
              protected_count[count] = Race
              count = count + 1
    # penalize
    index = []
    for i in range(32):
      cor = pearsonr(protected_count,hidden_feature_count[:,i])
      torch.FloatTensor.abs_(cor)
      if np.abs(cor) > threshold:
        index.append(i)
    net.Penalize(torch.tensor(index))

In [0]:
# threshold can be set from 0 ~ 0.14
threshold = 0.14
penalize_model_cor(train_loader, seq_cnn, threshold)
test_model(val_loader, seq_cnn, loss_function, fairmasking=True)

In [0]:
seq_cnn.Recover()

### Logistic regression based Masking


Another approach we propose is to fit a logistic regression using hidden features as regressors and ethnicity as outcome variable. Since there are 7 classes in total, there will be 6 coefficients for each of the hidden features. We calculate the difference between maximum and minimum coefficient values for each feature. We regard this value as a measure of dependence between the corresponding hidden feature and protected feature (ethnicity), since if the feature is completely independently with protected feature, then all 6 coefficients should be the same (equal to 1). The larger the difference, the more different contributions the hidden feature makes to predict each class (of protected feature). We rank all the hidden features according to this difference. We can then set threshold to mask out the top $k$ features.

In [0]:
def penalize_model_lr(train_dataloader, net, threshold):
    total = train_gender.shape[0]
    hidden_feature_count = np.zeros([total,32])
    protected_count = np.zeros(total)
    count = 0
    for i, (data, race, gender) in enumerate(train_dataloader):
            data, race, gender = data.to(device), race.to(device), gender.to(device)
            output,hidden_feature = net(data)

            for j in range(data.shape[0]):
              H_feature = Variable(hidden_feature[j],requires_grad = False)

              hidden_feature_count[count,:] = H_feature.cpu().numpy()
              protected_count[count] = race[j].float().to(device)
              count += 1

    hidden_feature_count = normalize(hidden_feature_count, axis=0)
    clf_lr = LogisticRegression(penalty = 'l2', max_iter = 10000, tol=1e-4, random_state=24, solver = 'lbfgs')
    clf_lr.fit(hidden_feature_count, protected_count)
    
    # report the training and test accuracy
    training_accuracy = clf_lr.score(hidden_feature_count, protected_count)
    pred = clf_lr.predict(hidden_feature_count)

    print ('training accuracy is ' + str(training_accuracy))
    params = clf_lr.coef_
    mn = np.amin(clf_lr.coef_,axis = 0)
    mx = np.amax(clf_lr.coef_,axis = 0)
    delta = mx - mn
    # rank the hidden features
    sortd = np.argsort(-delta)

    # penalize threshold = 3 is good
    index = []
    for i in range(threshold):
        index.append(sortd[i])
    
    net.Penalize(torch.tensor(index))

In [0]:
# can set threshold from 1 to 32
threshold = 5
penalize_model_lr(train_loader,seq_cnn,threshold)
test_model(val_loader,seq_cnn,loss_function, fairmasking=True)

# Section 5: Transfer Learning

The following code tests the IMDb data on the pretrained resnet model from part 3D.

In [0]:
loss_function = nn.CrossEntropyLoss()
test_model(test_loader, resnet, loss_function)

#Section 6: Analysis of Outputs

This section documents the extreme class imbalance of ethnicity in the IMDb dataset, and slight class imbalance of gender, by plotting grouped bar charts for each feature.

In [0]:
labels = ['Black','EastAsian','Indian','Latino','MiddleEastern','SoutheastAsian','White']

tr = Counter(train_label.race).most_common()
v = Counter(val_label.race).most_common()
te = Counter(test_label.ethnicity).most_common()

x = np.arange(len(labels))

train_freq = [0] * 7
val_freq = [0] * 7
test_freq = [0] * 7

for i in range(7):
    tr_tup = tr[i]
    v_tup = v[i]
    te_tup = te[i]
    
    train_freq[labels.index(tr_tup[0])] = (tr_tup[1]/86744) * 100
    val_freq[labels.index(v_tup[0])] = (v_tup[1]/10954) * 100
    test_freq[labels.index(te_tup[0])] = (te_tup[1]/347701) * 100

# the label locations
width = 0.30  # the width of the bars

fig, ax = plt.subplots(figsize=(15,10))
rects1 = ax.bar(x - width, train_freq, width, label='FairFace (train)')
rects2 = ax.bar(x, val_freq, width, label='FairFace (val)')
rects3 = ax.bar(x + width, test_freq, width, label='IMDB (test)')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_title('Frequency (%) of ethnicity label in each dataset',fontsize=25)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(fontsize=18)
ax.tick_params(labelsize=15)


def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        height = round(height, 2)
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()

In [0]:
labels = ['Female','Male']

tr = Counter(train_label.gender).most_common()
v = Counter(val_label.gender).most_common()
te = Counter(test_label.gender).most_common()

x = np.arange(len(labels))

train_freq = [0] * 2
val_freq = [0] * 2
test_freq = [0] * 2

for i in range(2):
    tr_tup = tr[i]
    v_tup = v[i]
    te_tup = te[i]
    
    train_freq[labels.index(tr_tup[0])] = (tr_tup[1]/86744) * 100
    val_freq[labels.index(v_tup[0])] = (v_tup[1]/10954) * 100
    test_freq[labels.index(te_tup[0])] = (te_tup[1]/347701) * 100

# the label locations
width = 0.30  # the width of the bars

fig, ax = plt.subplots(figsize=(6,7.5))
rects1 = ax.bar(x - width, train_freq, width, label='FairFace (train)')
rects2 = ax.bar(x, val_freq, width, label='FairFace (val)')
rects3 = ax.bar(x + width, test_freq, width, label='IMDB (test)')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_title('Frequency (%) of gender label in each dataset',fontsize=15)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(fontsize=12)
ax.tick_params(labelsize=10)


def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        height = round(height, 2)
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()